In [1]:
!pip install tvm_valuetypes

In [2]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM
import codecs
from tvm_valuetypes import Cell

# Easy test examples

In [3]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Use code: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Load cp0
[ 4][t 0][2022-10-20 01:59:32.377258][arithops.cpp:34]	execute PUSHINT 1

[ 4][t 0][2022-10-20 01:59:32.377415][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0
Got 1 from TVM: 1


In [4]:
tvm_result

[1]

In [5]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()
tvm_result

[{'type': 'continuation',
  'value': 'te6ccuEBAgEADgASHAEMBAAAAAYAAQAGce1DB7kzoA=='},
 1]

In [6]:
cell = Cell()
b64 = tvm_result[0]['value']
cell.data.from_bytes(codecs.decode(codecs.encode(b64, 'utf8'), 'base64'))

cell.data # This is continuation

b'\xb5\xee\x9cr\xe1\x01\x02\x01\x00\x0e\x00\x12\x1c\x01\x0c\x04\x00\x00\x00\x06\x00\x01\x00\x06q\xedC\x07\xb93\xa0'

In [7]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

[124711402,
 0,
 0,
 0,
 0,
 0,
 0,
 [100000, None],
 {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
 None]

## Change c7

In [8]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [9]:
tvm.set_c7(unixtime=179, 
           randseed=100, 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=100000000)

In [10]:
tvm.run_vm()

[[124711402,
  0,
  0,
  179,
  0,
  0,
  100,
  [100000000, None],
  {'type': 'cellSlice',
   'value': 'te6ccuEBAQEAJABIAEOADVySWN59rHc58b7yQKDOcZsaNW4TYZQKwLQsP8G0qTtQ5qaveg=='},
  None]]

## Get detailed result

In [11]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,
"\nVM success: ",tvm.success)

Exit code:  -1 
C5:  te6ccuEBAQEAAgAEAABmLc6k 
Gas credit:  0 
Gas used:  31 
C4:  te6ccuEBAQEAAgAEAABmLc6k 
VM Steps:  2 
VM final hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM init hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM success:  True


## Set stack

In [18]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H", log_level=6)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C


In [19]:
# actually python3 & TON have really big integers :))
tvm.set_stack([
    190009329293929392939239293293929329392949949949299008888888880999999888, 
    {'type': "cell",
     'value': 'te6ccgEBAQEABAAABO1H'}, 
    [{'type': "cellSlice",
     'value': 'te6ccgEBAQEABAAABO1H'},
     12312312313213132,
     [1231232132, 1321231231, 57, 179]
    ]])

got value:  190009329293929392939239293293929329392949949949299008888888880999999888
got value:  {'type': 'cell', 'value': 'te6ccgEBAQEABAAABO1H'}
got value:  [{'type': 'cellSlice', 'value': 'te6ccgEBAQEABAAABO1H'}, 12312312313213132, [1231232132, 1321231231, 57, 179]]


In [20]:
tvm.run_vm()

DEBUG: Use code: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C
DEBUG: Load cp0
[ 4][t 0][2022-10-20 02:00:05.233950][contops.cpp:694]	execute PUSH c7

[ 4][t 0][2022-10-20 02:00:05.234126][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0
DEBUG: Parse stack item #1
DEBUG: Parse stack item #2
DEBUG: Parse stack item #3


[[124711402,
  0,
  0,
  0,
  0,
  0,
  0,
  [100000, None],
  {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
  None],
 [{'type': 'cellSlice', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
  12312312313213132,
  [1231232132, 1321231231, 57, 179]],
 {'type': 'cell', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
 190009329293929392939239293293929329392949949949299008888888880999999888]